In [1]:
import numpy as np
import pandas as pd
from textrank4zh import TextRank4Keyword, TextRank4Sentence
import jieba
import jieba.analyse
import jieba.posseg as pseg
import re
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import datetime
from sklearn.cluster import KMeans
from sklearn import preprocessing
import time

In [2]:
##read data
file_1 = pd.read_csv("C:\\Users\\ading\\Desktop\\Main Folder\\Avalon Ding\\Main folder\\2019-20\\Sentimental Analysis\\nba_weibo_2019_1123_pm.csv", index_col = 0)
file_2 = pd.read_csv("C:\\Users\\ading\\Desktop\\Main Folder\\Avalon Ding\\Main folder\\2019-20\\Sentimental Analysis\\nba_weibo_2019_1124_am.csv", index_col = 0)

In [3]:
##data cleansing
#file = file_1
file = pd.concat((file_1, file_2), axis = 0)

file.drop(file.loc[file['UserID'] == 1883881851, :].index, inplace = True)

index_list = []
for x in file.WeiboID:
    if x == '4435958336617619':
        index_list.append(file.loc[file['Content'] == x, :].index[0])

# drop重复内容行
file.drop_duplicates(subset='Content', keep='first', inplace = True)
print(file.shape)

# 按关键词drop行
index_list = []
for x in file.Content:
    if ("叶落" in x) | ("买球" in x) | ("滚球" in x):
        index_list.append(file.loc[file['Content'] == x,:].index[0])

file.drop(index_list, axis = 0, inplace= True)
print(file.shape)

def clean_data(row):
        x = row['Content']
        # 去掉hastag和包含内容
        # row_num=file_1.loc[file_1['Content'] == '{}'.format(x) ,:].index
        if "#" in x:
            hashtag = re.findall(r'\#[^#]*\#', x)
            x = re.sub(r'\#[^#]*\#', '', x)
            x = re.sub(r'http:.* ', '', x)
            row['Content'] = x
            row['hashtag'] = hashtag
        # 去掉[]和包含内容
        if "[" in x:
            row['Content'] = re.sub(r'\[[^)]*\]', '', x)

        '''                                                                                           
            # 查看#移除情况
            index_list_1 = []
            for x in file_1.Content:
                if "#" in x:
                    index_list_1.append(file_1.loc[file_1['Content'] == x, :].index[0])
            print(file_1.loc[index_list_1, "Content"].count())
            print("-" * 10)
            # 查看[]移除情况
            index_list_2 = []
            for x in file_1.Content:
                if "[" in x:
                    index_list_2.append(file_1.loc[file_1['Content'] == x, :].index[0])
            print(file_1.loc[index_list_2, "Content"].count())
            print("-" * 10)
        '''
        return row

file = file.apply(clean_data, axis=1)

(2145, 6)
(2084, 6)


In [4]:
#整合所有内容
weibo_keywords = ''
for content in file.Content:
    weibo_keywords = weibo_keywords + ' ' + '{}'.format(content)
    
file_content = str.encode(weibo_keywords)
#分词
seg_list = jieba.cut(file_content, cut_all=False)
#print(' '.join(seg_list))
#print('-'*100)
# 获取分词和词性
words = pseg.cut(file_content)
#for word, flag in words:
    #print('%s, %s' % (word, flag))
#print('-'*100)

# 通过TF-IDF获取关键词
keywords = jieba.analyse.extract_tags(file_content, topK=40, withWeight=True, allowPOS=('n','nr','ns'))
for item in keywords:
    print(item[0],item[1])

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\ading\AppData\Local\Temp\jieba.cache
Loading model cost 0.870 seconds.
Prefix dict has been built succesfully.


湖人 0.3119545285345184
篮板 0.1996407747309457
詹姆斯 0.16344907016802218
马刺 0.13920026324944543
热火 0.11555867409246935
球员 0.11235842555888383
公牛 0.11159905164009923
老鹰 0.09951393434277585
球队 0.09625618932549972
黄蜂 0.09516118369719789
森林狼 0.09424141532551662
猛龙 0.09239039089490368
头条 0.09103369463579684
篮球 0.08303049008773146
尼克斯 0.08259440937779333
连胜 0.07792660514092235
主场 0.07372071047155165
爵士 0.07295946865392644
雷霆 0.07245102651314886
鹈鹕 0.07080920568227088
拉文 0.07068837234500876
雄鹿 0.06446392927047284
集锦 0.06291265138705779
客场 0.06214766087990252
快船 0.05959745717205079
全场 0.05706765598831699
文章 0.05683854051517688
骑士 0.053432823167478105
步行者 0.053016279258756564
伦纳德 0.052429192764460016
太阳 0.05162665529812376
险胜 0.05094559414545826
独行侠 0.050072293014658494
赛事 0.0494880108447875
本场 0.04796285943467601
生涯 0.04770908838432866
魔术 0.04655900825047052
三分球 0.04376915493925861
浓眉 0.04364646243104204
勇士 0.04303817562731349


In [7]:
#!/usr/bin/env python
# coding: utf-8

##file 1 sentiment data cleansing and modeling

from aip import AipNlp

APP_ID = '3dd8a9e3daf84f018c7a4d6a26675c22'
API_KEY = '621c4ef4ca2247f39560831311e067c4'
SECRET_KEY = 'e4bf7e8686414d2baaf484e436a91ec4'

client = AipNlp(APP_ID, API_KEY, SECRET_KEY)

# 去掉重复行
file_1.drop_duplicates(subset='Content', keep='first', inplace=True)
print(file_1.shape)

# 去掉两个关键字（来自于营销号）
index_list = []
for x in file_1.Content:
    if ("叶落" in x) | ("买球" in x) | ("滚球" in x):
        index_list.append(file_1.loc[file_1['Content'] == x, :].index[0])

file_1.drop(index_list, axis=0, inplace=True)
print(file_1.shape)

#去掉hastag和包含内容
for x in file_1.Content:
    if "#" in x:
        file_1.loc[file_1.loc[file_1['Content'] == '{}'.format(x) ,:].index,"Content"] = re.sub(r'\#[^#]*\#','',x)

#去掉[]和包含内容
for x in file_1.Content:
    if "[" in x:
        file_1.loc[file_1.loc[file_1['Content'] == '{}'.format(x) ,:].index,"Content"] = re.sub(r'\[[^)]*\]','', x)

#查看#移除情况
index_list_1 = []
for x in file_1.Content:
    if "#" in x:
        index_list_1.append(file_1.loc[file_1['Content'] == x, :].index[0])
        
print(file_1.loc[index_list_1,"Content"].count())
print("-"*10)

#查看[]移除情况
index_list_2 = []
for x in file_1.Content:
    if "[" in x:
        index_list_2.append(file_1.loc[file_1['Content'] == x, :].index[0])
        
print(file_1.loc[index_list_2,"Content"].count())
print("-"*10)


file_1 = file_1.loc[:,['Content','DateID']]

print(file_1.shape)

#reset index
file_1 = file_1.reset_index(drop = True)

#create情感评分函数
def get_score(row):
    content=row['Content']
    if len(content)==0:
        error_msg="content empty"
        row['sentiment_score']=error_msg
        row['positive_prob']=error_msg
        row['negative_prob']=error_msg
        return row
    # print(len(file_1.loc[file_1['Content'] == x,:]))
    # print(file_1.loc[file_1['Content'] == x,:].index)
    # print("{0:.2%}".format(file_1.loc[file_1['Content'] == x,:].index[0]/file_1.tail(1).index[0]))
    x = client.sentimentClassify(str(content).encode(encoding='gbk', errors='ignore').decode('gbk'))
    if 'items' in x.keys():
        # print(x['items'][0]['sentiment'])
        row['sentiment_score']=x['items'][0]['sentiment']
        row['positive_prob'] = x['items'][0]['positive_prob']
        row['negative_prob'] = x['items'][0]['negative_prob']
        print(row['sentiment_score'])
        return row
    else:
        if ('error_code' in x.keys()) and (x['error_code'] in [4,17,18,19]):
            print(x)
            time.sleep(4)
            return get_score(row)
        else:
            error_msg = x
            row['sentiment_score'] = error_msg
            row['positive_prob'] = error_msg
            row['negative_prob'] = error_msg
            return row

#在Dataframe中增加相应的特征列
file_1 = file_1.apply(get_score, axis=1)

#save as excel
file_1.to_excel('C:\\Users\\ading\\Desktop\\Main Folder\\Avalon Ding\\Main folder\\2019-20\\Sentimental Analysis\\Sentimental report\\Experiment\\Sentiment_prob_{}.xlsx'.format(file_1.iloc[1][1]))

(1628, 2)
(1628, 2)
10
----------
0
----------
(1628, 2)
2
2
{'error_code': 18, 'error_msg': 'Open api qps request limit reached'}
2
{'error_code': 18, 'error_msg': 'Open api qps request limit reached'}
0
2
{'error_code': 18, 'error_msg': 'Open api qps request limit reached'}
0
{'error_code': 18, 'error_msg': 'Open api qps request limit reached'}
2
2
{'error_code': 18, 'error_msg': 'Open api qps request limit reached'}
0
2
0
2
2
{'error_code': 18, 'error_msg': 'Open api qps request limit reached'}
2
{'error_code': 18, 'error_msg': 'Open api qps request limit reached'}
2
2
0
{'error_code': 18, 'error_msg': 'Open api qps request limit reached'}
2
2
0
2
2
2
{'error_code': 18, 'error_msg': 'Open api qps request limit reached'}
2
2
{'error_code': 18, 'error_msg': 'Open api qps request limit reached'}
2
2
2
2
1
0
{'error_code': 18, 'error_msg': 'Open api qps request limit reached'}
2
2
{'error_code': 18, 'error_msg': 'Open api qps request limit reached'}
2
0
2
2
{'error_code': 18, 'error_msg

2
{'error_code': 18, 'error_msg': 'Open api qps request limit reached'}
2
2
{'error_code': 18, 'error_msg': 'Open api qps request limit reached'}
2
0
{'error_code': 18, 'error_msg': 'Open api qps request limit reached'}
0
2
{'error_code': 18, 'error_msg': 'Open api qps request limit reached'}
2
2
{'error_code': 18, 'error_msg': 'Open api qps request limit reached'}
2
{'error_code': 18, 'error_msg': 'Open api qps request limit reached'}
2
{'error_code': 18, 'error_msg': 'Open api qps request limit reached'}
2
{'error_code': 18, 'error_msg': 'Open api qps request limit reached'}
2
2
{'error_code': 18, 'error_msg': 'Open api qps request limit reached'}
2
2
{'error_code': 18, 'error_msg': 'Open api qps request limit reached'}
2
{'error_code': 18, 'error_msg': 'Open api qps request limit reached'}
0
2
{'error_code': 18, 'error_msg': 'Open api qps request limit reached'}
2
2
{'error_code': 18, 'error_msg': 'Open api qps request limit reached'}
2
2
{'error_code': 18, 'error_msg': 'Open api qp

1
2
{'error_code': 18, 'error_msg': 'Open api qps request limit reached'}
1
2
{'error_code': 18, 'error_msg': 'Open api qps request limit reached'}
2
2
{'error_code': 18, 'error_msg': 'Open api qps request limit reached'}
0
2
2
0
{'error_code': 18, 'error_msg': 'Open api qps request limit reached'}
2
2
{'error_code': 18, 'error_msg': 'Open api qps request limit reached'}
0
2
{'error_code': 18, 'error_msg': 'Open api qps request limit reached'}
2
2
2
2
2
2
2
0
{'error_code': 18, 'error_msg': 'Open api qps request limit reached'}
2
2
2
0
2
1
{'error_code': 18, 'error_msg': 'Open api qps request limit reached'}
2
2
2
2
{'error_code': 18, 'error_msg': 'Open api qps request limit reached'}
2
2
{'error_code': 18, 'error_msg': 'Open api qps request limit reached'}
1
2
2
2
{'error_code': 18, 'error_msg': 'Open api qps request limit reached'}
2
1
{'error_code': 18, 'error_msg': 'Open api qps request limit reached'}
2
2
0
2
{'error_code': 18, 'error_msg': 'Open api qps request limit reached'}
0


{'error_code': 18, 'error_msg': 'Open api qps request limit reached'}
2
2
2
2
{'error_code': 18, 'error_msg': 'Open api qps request limit reached'}
2
2
2
2
2
2
{'error_code': 18, 'error_msg': 'Open api qps request limit reached'}
2
1
2
2
{'error_code': 18, 'error_msg': 'Open api qps request limit reached'}
2
2
2
1
{'error_code': 18, 'error_msg': 'Open api qps request limit reached'}
1
2
{'error_code': 18, 'error_msg': 'Open api qps request limit reached'}
2
2
{'error_code': 18, 'error_msg': 'Open api qps request limit reached'}
0
2
{'error_code': 18, 'error_msg': 'Open api qps request limit reached'}
2
2
{'error_code': 18, 'error_msg': 'Open api qps request limit reached'}
2
2
{'error_code': 18, 'error_msg': 'Open api qps request limit reached'}
2
0
2
2
2
2
{'error_code': 18, 'error_msg': 'Open api qps request limit reached'}
0
0
0
2
2
2
{'error_code': 18, 'error_msg': 'Open api qps request limit reached'}
2
1
{'error_code': 18, 'error_msg': 'Open api qps request limit reached'}
2
2
{'

2
2
{'error_code': 18, 'error_msg': 'Open api qps request limit reached'}
2
2
{'error_code': 18, 'error_msg': 'Open api qps request limit reached'}
2
1
{'error_code': 18, 'error_msg': 'Open api qps request limit reached'}
2
2
{'error_code': 18, 'error_msg': 'Open api qps request limit reached'}
2
2
0
2
2
2
2
2
{'error_code': 18, 'error_msg': 'Open api qps request limit reached'}
2
2
0
2
2
2
{'error_code': 18, 'error_msg': 'Open api qps request limit reached'}
2
2
{'error_code': 18, 'error_msg': 'Open api qps request limit reached'}
0
0
{'error_code': 18, 'error_msg': 'Open api qps request limit reached'}
2
2
{'error_code': 18, 'error_msg': 'Open api qps request limit reached'}
0
2
{'error_code': 18, 'error_msg': 'Open api qps request limit reached'}
1
2
{'error_code': 18, 'error_msg': 'Open api qps request limit reached'}
2
0
2
2
2
2
0
0
{'error_code': 18, 'error_msg': 'Open api qps request limit reached'}
2
0
{'error_code': 18, 'error_msg': 'Open api qps request limit reached'}
2
2
{'

{'error_code': 18, 'error_msg': 'Open api qps request limit reached'}
0
0
2
2
{'error_code': 18, 'error_msg': 'Open api qps request limit reached'}
0
2
{'error_code': 18, 'error_msg': 'Open api qps request limit reached'}
2
2
{'error_code': 18, 'error_msg': 'Open api qps request limit reached'}
2
2
{'error_code': 18, 'error_msg': 'Open api qps request limit reached'}
0
2
2
2
{'error_code': 18, 'error_msg': 'Open api qps request limit reached'}
2
1
{'error_code': 18, 'error_msg': 'Open api qps request limit reached'}
2
1
2
2
{'error_code': 18, 'error_msg': 'Open api qps request limit reached'}
2
2
{'error_code': 18, 'error_msg': 'Open api qps request limit reached'}
2
0
{'error_code': 18, 'error_msg': 'Open api qps request limit reached'}
0
2
{'error_code': 18, 'error_msg': 'Open api qps request limit reached'}
2
2
{'error_code': 18, 'error_msg': 'Open api qps request limit reached'}
2
2
{'error_code': 18, 'error_msg': 'Open api qps request limit reached'}
2
2
{'error_code': 18, 'error_

In [6]:
#!/usr/bin/env python
# coding: utf-8

##file 2 sentiment data cleansing and modeling

from aip import AipNlp

APP_ID = '3dd8a9e3daf84f018c7a4d6a26675c22'
API_KEY = '621c4ef4ca2247f39560831311e067c4'
SECRET_KEY = 'e4bf7e8686414d2baaf484e436a91ec4'

client = AipNlp(APP_ID, API_KEY, SECRET_KEY)

# 去掉重复行
file_2.drop_duplicates(subset='Content', keep='first', inplace=True)
print(file_2.shape)

# 去掉两个关键字（来自于营销号）
index_list = []
for x in file_2.Content:
    if ("叶落" in x) | ("买球" in x) | ("滚球" in x):
        index_list.append(file_2.loc[file_2['Content'] == x, :].index[0])

file_2.drop(index_list, axis=0, inplace=True)
print(file_2.shape)

#去掉hastag和包含内容
for x in file_2.Content:
    if "#" in x:
        file_2.loc[file_2.loc[file_2['Content'] == '{}'.format(x) ,:].index,"Content"] = re.sub(r'\#[^#]*\#','',x)

#去掉[]和包含内容
for x in file_2.Content:
    if "[" in x:
        file_2.loc[file_2.loc[file_2['Content'] == '{}'.format(x) ,:].index,"Content"] = re.sub(r'\[[^)]*\]','', x)

#查看#移除情况
index_list_1 = []
for x in file_2.Content:
    if "#" in x:
        index_list_1.append(file_2.loc[file_2['Content'] == x, :].index[0])
        
print(file_2.loc[index_list_1,"Content"].count())
print("-"*10)

#查看[]移除情况
index_list_2 = []
for x in file_2.Content:
    if "[" in x:
        index_list_2.append(file_2.loc[file_2['Content'] == x, :].index[0])
        
print(file_2.loc[index_list_2,"Content"].count())
print("-"*10)


file_2 = file_2.loc[:,['Content','DateID']]

print(file_2.shape)

#reset index
file_2 = file_2.reset_index(drop = True)

#create情感评分函数
def get_score(row):
    content=row['Content']
    if len(content)==0:
        error_msg="content empty"
        row['sentiment_score']=error_msg
        row['positive_prob']=error_msg
        row['negative_prob']=error_msg
        return row
    # print(len(file_1.loc[file_1['Content'] == x,:]))
    # print(file_1.loc[file_1['Content'] == x,:].index)
    # print("{0:.2%}".format(file_1.loc[file_1['Content'] == x,:].index[0]/file_1.tail(1).index[0]))
    x = client.sentimentClassify(str(content).encode(encoding='gbk', errors='ignore').decode('gbk'))
    if 'items' in x.keys():
        # print(x['items'][0]['sentiment'])
        row['sentiment_score']=x['items'][0]['sentiment']
        row['positive_prob'] = x['items'][0]['positive_prob']
        row['negative_prob'] = x['items'][0]['negative_prob']
        print(row['sentiment_score'])
        return row
    else:
        if ('error_code' in x.keys()) and (x['error_code'] in [4,17,18,19]):
            print(x)
            time.sleep(4)
            return get_score(row)
        else:
            error_msg = x
            row['sentiment_score'] = error_msg
            row['positive_prob'] = error_msg
            row['negative_prob'] = error_msg
            return row

#在Dataframe中增加相应的特征列
file_2 = file_2.apply(get_score, axis=1)

#save as excel
file_2.to_excel('C:\\Users\\ading\\Desktop\\Main Folder\\Avalon Ding\\Main folder\\2019-20\\Sentimental Analysis\\Sentimental report\\Experiment\\Sentiment_prob_{}.xlsx'.format(file_2.iloc[1][1]))

(532, 6)
(529, 6)
3
----------
0
----------
(529, 2)
2
2
{'error_code': 18, 'error_msg': 'Open api qps request limit reached'}
2
{'error_code': 18, 'error_msg': 'Open api qps request limit reached'}
1
0
{'error_code': 18, 'error_msg': 'Open api qps request limit reached'}
2
{'error_code': 18, 'error_msg': 'Open api qps request limit reached'}
2
2
{'error_code': 18, 'error_msg': 'Open api qps request limit reached'}
2
1
{'error_code': 18, 'error_msg': 'Open api qps request limit reached'}
2
1
{'error_code': 18, 'error_msg': 'Open api qps request limit reached'}
1
2
{'error_code': 18, 'error_msg': 'Open api qps request limit reached'}
2
0
{'error_code': 18, 'error_msg': 'Open api qps request limit reached'}
0
{'error_code': 18, 'error_msg': 'Open api qps request limit reached'}
2
2
{'error_code': 18, 'error_msg': 'Open api qps request limit reached'}
2
2
{'error_code': 18, 'error_msg': 'Open api qps request limit reached'}
2
2
{'error_code': 18, 'error_msg': 'Open api qps request limit r

2
2
{'error_code': 18, 'error_msg': 'Open api qps request limit reached'}
2
0
{'error_code': 18, 'error_msg': 'Open api qps request limit reached'}
2
2
{'error_code': 18, 'error_msg': 'Open api qps request limit reached'}
2
{'error_code': 18, 'error_msg': 'Open api qps request limit reached'}
0
2
2
{'error_code': 18, 'error_msg': 'Open api qps request limit reached'}
2
{'error_code': 18, 'error_msg': 'Open api qps request limit reached'}
0
{'error_code': 18, 'error_msg': 'Open api qps request limit reached'}
1
{'error_code': 18, 'error_msg': 'Open api qps request limit reached'}
2
2
{'error_code': 18, 'error_msg': 'Open api qps request limit reached'}
0
2
{'error_code': 18, 'error_msg': 'Open api qps request limit reached'}
2
1
{'error_code': 18, 'error_msg': 'Open api qps request limit reached'}
2
2
{'error_code': 18, 'error_msg': 'Open api qps request limit reached'}
1
0
{'error_code': 18, 'error_msg': 'Open api qps request limit reached'}
0
2
{'error_code': 18, 'error_msg': 'Open ap

0
1
2
{'error_code': 18, 'error_msg': 'Open api qps request limit reached'}
0
2
{'error_code': 18, 'error_msg': 'Open api qps request limit reached'}
1
{'error_code': 18, 'error_msg': 'Open api qps request limit reached'}
1
2
{'error_code': 18, 'error_msg': 'Open api qps request limit reached'}
0
{'error_code': 18, 'error_msg': 'Open api qps request limit reached'}
0
2
{'error_code': 18, 'error_msg': 'Open api qps request limit reached'}
2
1
{'error_code': 18, 'error_msg': 'Open api qps request limit reached'}
2
{'error_code': 18, 'error_msg': 'Open api qps request limit reached'}
2
2
{'error_code': 18, 'error_msg': 'Open api qps request limit reached'}
2
0
{'error_code': 18, 'error_msg': 'Open api qps request limit reached'}
2
2
{'error_code': 18, 'error_msg': 'Open api qps request limit reached'}
0
2
{'error_code': 18, 'error_msg': 'Open api qps request limit reached'}
2
2
{'error_code': 18, 'error_msg': 'Open api qps request limit reached'}
0
2
{'error_code': 18, 'error_msg': 'Open 

In [8]:
#combine two datasets
file = pd.concat((file_1, file_2), axis = 0)
date = '{}-{}-{}'.format(file.iloc[-1][1].split('-')[0],file.iloc[-1][1].split('-')[1],file.iloc[-1][1].split('-')[2])
#add one column to specify the date
file['date'] = date

In [9]:
file.sentiment_score.value_counts()[:3]/(file.sentiment_score.value_counts()[:3].sum())

2    0.724570
0    0.191361
1    0.084069
Name: sentiment_score, dtype: float64

In [7]:
#perform clustering
data = file[["positive_prob","negative_prob"]]
kmeans = KMeans(n_clusters=3)

# 规范化到 [0,1] 空间
min_max_scaler=preprocessing.MinMaxScaler()
data=min_max_scaler.fit_transform(data)

#file.reset_index(drop = True, inplace = True)

# kmeans 算法
kmeans.fit(data)
predict_y = kmeans.predict(data)
# 合并聚类结果，插入到原数据中
result = pd.concat((file,pd.DataFrame(predict_y)),axis=1)
result.rename({0:u'聚类结果'},axis=1,inplace=True)
#print(result)
# 将结果导出到CSV文件中
result.to_excel("C:\\Users\\ading\\Desktop\\Main Folder\\Avalon Ding\\Main folder\\2019-20\\Sentimental Analysis\\Sentimental report\\Raw data\\sentiment_cluster_{}.xlsx".format(date))

TypeError: float() argument must be a string or a number, not 'dict'